In [ ]:
# %% allow reloading of modules
%load_ext autoreload
%autoreload 2

In [ ]:
# %% Initilizing
import pandas as pd
import numpy as np
import os
path_parent = os.path.dirname(os.getcwd())
data_directory = os.path.join(path_parent, "data")
os.chdir(path_parent)
pd.options.display.max_columns = None

In [ ]:
import pycrash
from pycrash.project import Project
from pycrash.vehicle import Vehicle
from pycrash.kinematicstwo import KinematicsTwo

In [ ]:

# TODO:

# %% Import Modules
import matplotlib.pyplot as plt
from matplotlib.pyplot import text
from src.functions import EnergyDV, SpringSeriesKeff
from src.project import Project, project_info, load_project
from src.vehicle import Vehicle
from src.kinematicstwo import KinematicsTwo
from scipy import signal
from scipy import integrate
import pandas as pd
import numpy as np
import pickle
import json

pd.options.display.max_columns = None
from IPython import get_ipython
from IPython.display import display
get_ipython().run_line_magic('matplotlib', 'inline')

## Create Project
- projects are used to store basic information about the project  
- name, type of impact, type of simulation to be run, description, notes  
- the project will be used to save all associated aspects, vehicles, simulations etc.  

In [ ]:
project_inputs = {'name':'Practice', 'pdesc':'single motion', 'sim_type':'SV', 'impact_type':'none',
                  'note':'single vehicle motion demo'}
proj = Project(project_inputs)

### Generate dataframe with driver inputs for vehicle 1 (striking vehicle)
- note - think of a dataframe as a basic excel sheet (header row and data columns)
- the time duration "end_time" is critical becuase it determines length of simulation

In [ ]:
end_time = 5  # 5 second simulation
t = list(np.arange(0, end_time+0.1, 0.1))  # create time array from 0 to end time from user
throttle = [0] * len(t)                                # no throttle
brake = [0] * len(t)                                   # no braking
steer = [0] * len(t)                                   # no steering
driver_input_dict = {'t':t, 'throttle':throttle, 'brake':brake, 'steer':steer}
driver_input_df = pd.DataFrame.from_dict(driver_input_dict)
print('Vehicle 1 Driver Inputs:')
driver_input_df.head() # first 5 rows of driver input data

### Create Vehicle 1:
- "Vehicle" stores information about a single vehicle - all possible inputs do not need to be entered
- creating a Vehicle requries a "name" which is used to identify the vehicle in outputs / plots etc. 

In [ ]:
vehicle_input_dict = {"year":2016,
"make":"Subaru",
"model":"WRX Sti",
"weight":3000,
"vin":"123abc",
"brake":0,
"steer_ratio":16.5,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"width":6.6,
"length":20.66,
"hcg":2,
"lcgf":5.6,
"lcgr":7.76,
"wb":13.36,
"track":5.7,
"f_hang":3.2,
"r_hang":3.873,
"tire_d":2.716666667,
"tire_w":0.866666667,
"izz":3711,
"fwd":0,
"rwd":1,
"awd":0,
"A":100,
"B":41,
"k":1000,
"L":0,
"c":0,
"vx_initial":5,
"vy_initial":0,
"omega_z":0}

veh1 = Vehicle('Veh1', vehicle_input_dict)
veh1.driver_input = driver_input_df

veh2 = Vehicle('Veh2', vehicle_input_dict)
veh2.driver_input = driver_input_df
veh2.init_x_pos = 20
veh2.init_y_pos = 30
veh2.head_angle = 270

### Create Vehicle 2
- vehicle data can also be created by importing a CSV file in the data/input directory
- `TODO`: change this directory to the "Project" directory - seperate input folder etc.

In [ ]:
veh2 = Vehicle('Veh2')
veh2.load_specs('fordGT.csv')

### driver input can be added at anytime

In [ ]:
end_time = 5  # 5 second simulation
t = list(np.arange(0, end_time+0.1, 0.1))  # create time array from 0 to end time from user
throttle = [0] * len(t)                                # no throttle
brake = [0] * len(t)                                   # no braking
steer = [0] * len(t)                                   # no steering
driver_input_dict = {'t':t, 'throttle':throttle, 'brake':brake, 'steer':steer}
driver_input_df = pd.DataFrame.from_dict(driver_input_dict)
print('Vehicle 2 Driver Inputs:')

veh2.driver_input = driver_input_df
veh2.driver_input.head() # first 5 rows of driver input data

### Create instance of two vehicle motion
- input requires 'name', 'impact_type', veh1, veh2
-  user will be prompted to assign impact location, direction of normal impact plane and impacting edge for struck vehicle

In [ ]:
ss1 = KinematicsTwo('run1', 'IMPC', veh1, veh2)

### set-up vehicle initial location
- use any prior simulation motion data to show paths

In [ ]:
ss1.initial_position()

### Run simulation

In [ ]:
print(f'Impacting Edge: {ss1.veh2.edgeimpact}')
print(f'Impacting Edge X Points: {ss1.veh2.edgeimpact_x1}, {ss1.veh2.edgeimpact_x2}')
print(f'Impacting Edge Y Points: {ss1.veh2.edgeimpact_y1}, {ss1.veh2.edgeimpact_y2}')

In [ ]:
from scipy import integrate
ss1.veh1.model['Dx'] = veh1.init_x_pos + integrate.cumtrapz(list(ss1.veh1.model.Vx), list(ss1.veh1.model.t), initial=0)

In [ ]:
ss1.veh1.model.head()

In [ ]:
ss1.simulate(impact_type = 'ss', ignore_driver = False)

In [ ]:
ss1.veh1.veh_model.head()

In [ ]:
ss1.veh2.veh_model.head()

### Plot Vehicle Motion

In [ ]:
ss1.draw_simulation(len(ss1.veh1.veh_model.t)-1)

In [ ]:
ss1.veh1.veh_model.head()

In [ ]:
max(ss1.veh1.veh_model.Fx)

In [ ]:
ss1.veh2.edgeimpact

In [ ]:
xy_ratio